In [ ]:
## plot_scope_traces_vibration_measurements_sept2017_EOM_linewidth
# 
# Measure cavity transmission under the influence of vibrations
# Use EOM to generate side-peaks of laser
# Record data using ossiloscope
#
# Manually select favourable time window where the peak with side-peaks is visible.
# Then fit three lorenzian functions to estimate both linewidth (width of main peak) and 
#   calibrate time versus frequency (from distance between side-peaks and known spacing in frequency = 2*EOM)
#
# 
# HansonLab / Team Diamond, Project Cavity
# Wouter Westerveld, 19 September 2017

In [ ]:
import numpy as np
import os.path
import pandas
import matplotlib
import scipy.signal
import scipy.constants

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec



## Load xviewer csv (from binary scope data) and scope csv

In [ ]:
def read_scope_xviewer_csv(filename):
    """ Read csv files exported using Yokogawa XViewer software
        
        Update 11-09-2017: auto-detect number of columns.        
    """
    headerRows = 10        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:
                i1 = line.find(',')+1
                i2 = line.find(',',i1)                
                timeResolution = float( line[i1:i2].strip(' \t\n\r') )        
                numberOfCols = line.count(',')    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows,usecols=range(1,numberOfCols)).values
    return data, timeResolution


def read_scope_csv(filename):
    """ Read csv files as stored by Yokogawa Ossiloscope.                     
    """    
    headerRows = 16        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.rfind(',')+1:-1].strip(' \t\n\r') )    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows).values[:,1:-1]
    return data, timeResolution



## Plot and Fit linewidth functions

In [ ]:
## LINEWIDTH IN GHz - FITTING AND ANALYSIS FUNCTIONS

import pylab
from scipy.optimize import leastsq # Levenberg-Marquadt Algorithm #

def lorentzian(x,p):    
    # Lorenzian function p[0] is center, p[1] is fwhm, p[2] is maximum intensity
    #    See http://mathworld.wolfram.com/LorentzianFunction.html
    #    But note that we not integrated intensity of one for p[2] = 1 but maxiumum intensity.
    return p[2] * (0.5 * p[1])**2 / ( ( x - p[0] )**2 + ( 0.5 * p[1] )**2 )

def lorentzian3(x,p):  
    return p[2] * ( lorentzian(x,[p[0], p[1], 1]) + lorentzian(x,p[3:6]) + lorentzian(x,p[6:9]) )

def fit_lorenzian(time,data,fwhmGuess,timeResolution):
    # initial guess
    a1Guess = np.amax(data)
    t1Guess = np.argmax(data) * timeResolution + time[0]
    # fit lorenzian function to data
    def residuals(p,y,x):
        return y - lorentzian(x,p)
    guessParameters = [t1Guess,fwhmGuess,a1Guess]  # [peak center, hwhm, intensity]
    pbest = leastsq(residuals,guessParameters,args=(data,time),full_output=1)
    fitParameters = pbest[0]
    return fitParameters, guessParameters

def fit_lorenzian3(time,data,fwhmGuess,timeDeltaGuess,timeResolution):
    # initial guess
    a1Guess = np.amax(data) * 0.7
    a2Guess = 1
    a3Guess = 1
    t1Guess = np.argmax(data) * timeResolution + time[0]
    t2Guess = t1Guess + timeDeltaGuess
    t3Guess = t1Guess - timeDeltaGuess
    # Fit three lorenzians to data.
    def residuals(p,y,x):
        return y - lorentzian3(x,p)
    guessParameters = np.array([[t1Guess, fwhmGuess, a1Guess], [t2Guess, fwhmGuess, a2Guess], [t3Guess, fwhmGuess, a3Guess]]).flatten()
    pbest = leastsq(residuals,guessParameters,args=(data,time),full_output=1)
    fitParameters = pbest[0]
    return fitParameters, guessParameters
    


def fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess):
    ## Fit linewith
    # Load data
    data, timeResolution = read_scope_csv(filename1)
    time = timeResolution * np.arange(data.shape[0]) 
    i1 = int( time1 / timeResolution )
    i2 = int( time2 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]
    # Fit
    transmissionFitParameters, transmissionGuessParameters = fit_lorenzian(time,data,fwhmGuess,timeResolution)
    # Plot
    fig = plt.figure(figsize=(16, 8))
    plt.subplot(121)    
    plt.plot( time, data )
    plt.plot( time, lorentzian(time,transmissionGuessParameters), linestyle=':' )
    plt.plot( time, lorentzian(time,transmissionFitParameters) )    
    print 'peak position {} s, fwhm {} s, intensity {} V'.format( \
        transmissionFitParameters[0], transmissionFitParameters[1], transmissionFitParameters[2] )

    ## Fit scanning speed with modulated signal 6 GHz
    # Load data
    data, timeResolution = read_scope_csv(filename2)
    time = timeResolution * np.arange(data.shape[0]) 
    i1 = int( time3 / timeResolution )
    i2 = int( time4 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]
    # Fit
    fitParameters, guessParameters = fit_lorenzian3(time,data,fwhmGuess,timeDeltaGuess,timeResolution)
    # Plot        
    plt.subplot(122)
    plt.plot( time, data )
    plt.plot( time, lorentzian3(time,guessParameters), linestyle=':' )
    plt.plot( time, lorentzian3(time,fitParameters) )
    plt.show()
        
    print 'peak positions (s)', fitParameters[[0,3,6]]
    print 'peak fwhm (s)', fitParameters[[1,4,7]]
    print 'peak intensity (V)', fitParameters[[2]], fitParameters[[2]] * fitParameters[[5]], \
         fitParameters[[2]] * fitParameters[[8]]
        
    dtSideModulationPeaks = np.amax( fitParameters[[0,3,6]] ) - np.amin( fitParameters[[0,3,6]] )
    dfOverDt = 12.0e9 / dtSideModulationPeaks    
    print 'df/dt = 12/{} GHz/s = {} THz/s'.format( \
        dtSideModulationPeaks, dfOverDt *1e-12 )
    cavityFwhmInS = transmissionFitParameters[1] * dfOverDt
    print 'cavity fwhm = {} s = {} GHz'.format( transmissionFitParameters[1], cavityFwhmInS*1e-9 )
    
    




## Measurements 1 9 2017: Red linewidth

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170901/scope'
fileName = "SCOPE_023.wdf.csv"  

data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
time = timeResolution * np.arange(data.shape[0]) 
print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )


In [ ]:
# Select regime of time-trace where the peak with side-bands are well separated.
# Between each peak with side-bands, signal should go to zero as much as possible.

timeStart  = 0.05595  # star time of selected window, s
timeWindow = 0.0008   # length of selected window, s

i1 = np.argmax(time>timeStart)
i2 = np.argmax(time>timeStart+timeWindow)
d = data[i1:i2].flatten()
t = time[i1:i2]

fig = plt.figure(figsize=(16, 8))
plt.plot(t, d )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 

In [ ]:
# smoothen data for peak-finding.
# dSmooth is smoothened version of d.

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filtfilt(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    print a.shape, b.shape
    y = filtfilt(b, a, data)
    return y

dSmooth = butter_lowpass_filtfilt(data=d, cutoff=0.01/timeResolution, fs=1.0/timeResolution)


fig = plt.figure(figsize=(16, 8))
plt.plot( t, d )
plt.plot( t, dSmooth )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 




In [ ]:
# Find peaks using local maximum.
# Data needs to be sufficiently smooth such that each local maximum corresponds to a peak.

peakMin = 0.35    # minimum voltage of peak, V
sideMin = 0.20    # minimum voltage of side-peak, V

localMaximumMask = np.r_[True, dSmooth[1:] > dSmooth[:-1]] & np.r_[dSmooth[:-1] > dSmooth[1:], True]

peakMask = localMaximumMask & (dSmooth > peakMin)
sideMask = localMaximumMask & (dSmooth > sideMin) & (dSmooth < peakMin)

peakInd = np.where( peakMask )[0]
sideInd = np.where( sideMask )[0]

fig = plt.figure(figsize=(16, 8))
plt.plot( t, d )
plt.plot( t[peakInd], d[peakInd] , '+')
plt.plot( t[sideInd], d[sideInd] , 'x')
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 




In [ ]:
# For each peak, fit three lorenzians
# 
# Find window
# Fit peaks
# Apply filtering rules
#  - maximum value at edge of window, V
#  - maximum asymmtery, maximum difference between time of left-side-peak to main-peak and time of right-side-peak to main-peak, translated to frequency, Hz
#

peakWin = 0.020e-3            # window of three peaks, s
peakWinEdgeMax = 0.10         # maximum value at edge of window, V
maxSidePeakAsymmetryHz = 1e9  # maximum asymmtery, maximum difference between time of left-side-peak to main-peak and time of right-side-peak to main-peak, translated to frequency, Hz
fwhmGuess = 5e-6              # guess FWHM of peaks
timeDeltaGuess = 5e-6         # guess distance between main peak and side-peaks
sideModulationFrequency = 8.0e9  # EOM modulation frequency, frequency difference between main peak and side peak, Hz

listFwhm = np.array([])

for i in peakInd:
    i1 = int( i - 0.5*peakWin/timeResolution )
    i2 = int( i + 0.5*peakWin/timeResolution )    
    dPeak = d[i1:i2]
    tPeak = t[i1:i2]

    fig = plt.figure(figsize=(16, 8))
    plt.plot( tPeak, dPeak )    
    plt.grid()
    plt.xlabel('time (s)')
    plt.ylabel('V')
    
    if (dPeak[0] > peakWinEdgeMax) | (dPeak[-1] > peakWinEdgeMax):
        print 'Signal at edge of window too high. Skipping.'
        plt.show()
        continue
    
    # Fit
    fitParameters, guessParameters = fit_lorenzian3(tPeak,dPeak,fwhmGuess,timeDeltaGuess,timeResolution)
    # Plot                    
    plt.plot( tPeak, lorentzian3(tPeak,guessParameters), linestyle=':' )
    plt.plot( tPeak, lorentzian3(tPeak,fitParameters) )
    plt.show()

    print 'fit'
    print 'peak positions (s)', fitParameters[[0,3,6]]
    print 'peak fwhm (s)', fitParameters[[1,4,7]]
    print 'peak intensity (V)', fitParameters[[2]], fitParameters[[2]] * fitParameters[[5]], \
         fitParameters[[2]] * fitParameters[[8]]

    dtSideModulationPeaks = np.amax( fitParameters[[0,3,6]] ) - np.amin( fitParameters[[0,3,6]] )
    dfOverDt = 2.0 * sideModulationFrequency / dtSideModulationPeaks    
    print 'df/dt = 2 * {}/{} GHz/s = {} THz/s'.format( sideModulationFrequency * 1e-9, dtSideModulationPeaks, dfOverDt *1e-12 )
    cavityFwhmInGhz = np.abs( fitParameters[1] * dfOverDt )
    print 'cavity fwhm = {} s = {} GHz'.format( fitParameters[1], cavityFwhmInGhz*1e-9 )    
    print 'sideband difference in GHz = {} GHz'.format( ( np.abs( fitParameters[3] - fitParameters[0] ) -  np.abs( fitParameters[6] - fitParameters[0] ) ) * dfOverDt *1e-9  )
    
    sidePeakAssymmetryHz = np.abs( ( np.abs( fitParameters[3] - fitParameters[0] ) -  np.abs( fitParameters[6] - fitParameters[0] ) ) ) * dfOverDt  
            
    if sidePeakAssymmetryHz > maxSidePeakAsymmetryHz:
        print 'Too large position assymetry between side-peaks. Skipping.'
        continue
    
    listFwhm = np.append( listFwhm, cavityFwhmInGhz )    
    

print 'number of peaks ', len(peakInd), ' ok fitted peaks ', len(listFwhm)
print 'mean fwhm GHz', np.mean( listFwhm ) *1e-9